# 💎 Bicameral Surgery: Gemma-3 (English Pruned)

**Strategy**: Instead of training from scratch, we perform 

## 1. Setup & Authentication
Gemma-3 requires a Hugging Face token.

In [ ]:
import os
from google.colab import userdata

# Set your HF_TOKEN in Colab Secrets
try:
    os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')
except:
    print("⚠️ Please set 'HF_TOKEN' in Colab Secrets or paste it below:")
    # os.environ["HF_TOKEN"] = "your_token_here"

In [ ]:
import torch
import gc

def cleanup():
    gc.collect()
    torch.cuda.empty_cache()
    print("🧹 GPU Memory Cleared")

cleanup()

%pip install -q torch transformers datasets tqdm wandb accelerate

## 2. Load and Prune Gemma-3
We strip the 256k vocabulary down to ~64k English-centric tokens.

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from surgery.prune import find_english_tokens, prune_gemma_model

MODEL_ID = "google/gemma-3-270m"

print("📥 Loading Base Gemma-3...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID, 
    torch_dtype=torch.bfloat16, 
    device_map="auto"
)

print("✂️ Pruning Multilingual Tokens...")
keep_indices = find_english_tokens(tokenizer, max_vocab_size=65536)
model = prune_gemma_model(base_model, keep_indices)

print(f"✅ Pruning Complete. New Vocab Size: {model.config.vocab_size}")

## 3. Perform Bicameral Surgery
Inject the Symbolic Lobe into the pre-trained blocks.

In [ ]:
from bicameral.config import BicameralConfig
from bicameral.surgery_model import BicameralSurgery

# Match Gemma-3 dims
config = BicameralConfig(
    d_model=model.config.hidden_size, # Should be 1024 for 270M
    n_roles=32,                       # Increased for reasoning
    d_filler=64,
    gate_init_bias=-2.0,              # Start with pure Gemma fluency
    vocab_size=model.config.vocab_size
)

model = BicameralSurgery(model, config)
print(f"🧠 Surgery Successful!")
print(f"Total Params: {model.count_total_parameters()/1e6:.1f}M")
print(f"Trainable (Symbolic) Params: {model.count_trainable_parameters()/1e6:.1f}M")

## 4. Logic Training (bAbI)
Focus compute entirely on reasoning induction.

In [ ]:
from data.dataset import LMDataset
from torch.utils.data import DataLoader
from torch.amp import GradScaler, autocast
from tqdm.notebook import tqdm

print("📦 Loading Logic Dataset (bAbI)...")
# We use the pruned model's vocab size but same tokenizer logic
ds_babi = LMDataset(tokenizer, "Muennighoff/babi", split="train", streaming=False)
loader = DataLoader(ds_babi, batch_size=8, shuffle=True)

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
scaler = GradScaler('cuda')
ORTHO_WEIGHT = 0.001
ACCUMULATION = 8

model.train()
pbar = tqdm(total=2000, desc="Logic Injection")

for step, batch in enumerate(loader):
    if step >= 2000: break
    
    input_ids = batch['input_ids'].to("cuda")
    
    with autocast('cuda'):
        out = model(input_ids, labels=input_ids)
        ortho_loss = model.get_orthogonality_loss() * ORTHO_WEIGHT
        loss = (out.loss + ortho_loss) / ACCUMULATION
    
    scaler.scale(loss).backward()
    
    if (step + 1) % ACCUMULATION == 0:
        scaler.step(optimizer)
        scaler.update()
        optimizer.zero_grad()
        pbar.set_postfix({'loss': f"{loss.item()*ACCUMULATION:.4f}"})
        pbar.update(1)
        
print("✨ Logic Training Complete!")

## 5. Final Reason-Fluency Test

In [ ]:
model.eval()
prompt = "John went to the kitchen. John picked up the ball. John went to the garden. John dropped the ball. Where is the ball?"

inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
with torch.no_grad():
    gen = model.gemma.generate(**inputs, max_new_tokens=20)
    
print(f"\nPrompt: {prompt}")
print(f"Gen: {tokenizer.decode(gen[0])}")